# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [218]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [698]:
%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - ((np.sum(np.hstack((l_c * l_c / l_s, r_c * r_c / r_s)),
                            axis=1)) / (l_s[0] + r_s[0]))

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(np.hstack((l_c * np.log2(l_c / l_s),
                                  r_c * np.log2(r_c / r_s)),
                                 axis=1))/(l_s[0] + r_s[0])

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c, axis=1) + np.max(r_c, axis=1)) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортируем значения признака х, y переставляется соответственоо
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        # Считаем минимальное количетство элементов выборки,
        # которые попадут влево и вправо при делении
        cut_size = np.int(self.min_samples_split / 2 - 1)
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        # Обрезаем массив, чтобы велево и вправо попало
        # как минимум cut_size элементов
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        # Нашли индексы элементов в которых изменился класс
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)
        # Проверяем случай, если разделение невозможно
        if len(r_border_ids) == 0:
            return np.inf, None
        # Считаем длины фрагментов массива на которых класс постоянен
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        # Создаем пустую матрицу.
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        # Заполняем ее ставя значение 1 на позиции ij где i
        # соответствует фрагменту где класс постоянен, а j номеру этого класса
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        # Умножаем нашу матрицу на  длинны блоков
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # Добавляем количество элементов каждого класса находящейся
        # в части sorted_y[:cut_size] в первый фрагмент
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)
        # в l_class_count лежит количество элементов каждого
        # класса (столбцы) до выхода из фрагмента (строки)
        # в r_class_count лежит количество элементов каждого
        # класса (столбцы) после выхода из фрагмента (строки)
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        # Считаем количество элементов каждого класса, в левом узле и
        # в правом узле если мы будет делать разбиение после каждого фрагмента
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Выбирает индекс элемента, по которому разделяем
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Возвращаем то что вычиается в формуле прироста информации и
        # разделяющую границу
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        classes = np.bincount(y, minlength=self.num_class) / x.shape[0]
        moda = np.argmax(classes)
        if pred_f == -1:
            self.feature_importances_ = np.zeros(x.shape[1])
            self.Size_fit = x.shape[0]
        if self.max_depth is None:
            self.max_depth = x.shape[0]
        if ((depth > self.max_depth) or (x.shape[0] < self.min_samples_split)
                or (classes[moda] > self.sufficient_share)):
            self.tree[node_id] = (self.LEAF_TYPE, moda, classes)
        else:
            feature_ids = self.get_feature_ids(x.shape[1])
            a = 1
            best_ind = -1
            best_thresh = -1
            for i in feature_ids:
                inp, thresh = self.__find_threshold(x[:, i], y)
                if (inp < a):
                    best_ind = i
                    best_thresh = thresh
                    a = inp
            if (best_ind == -1):
                self.tree[node_id] = (self.LEAF_TYPE, moda, classes)
            else:
                x_left, x_right, y_left, y_right =\
                    self.__div_samples(x, y, best_ind, best_thresh)
                if ((x_left.shape[0] == 0) or (x_right.shape[0] == 0)):
                    self.tree[node_id] = (self.LEAF_TYPE, moda, classes)
                else:
                    self.tree[node_id] = (self.NON_LEAF_TYPE,
                                          best_ind, best_thresh)
                    if self.G_function == self.__gini:
                        b = 1 - np.sum(classes ** 2)
                    elif self.G_function == self.__entropy:
                        b = -np.sum(classes * np.log2(classes))
                    else:
                        b = 1 - np.max(classes)
                    c = (b - a) * x.shape[0]
                    self.feature_importances_[best_ind] += (c / self.Size_fit)
                    new_id = 2 * node_id
                    self.__fit_node(x_left, y_left, new_id + 1, depth + 1, 0)
                    self.__fit_node(x_right, y_right, new_id + 2, depth + 1, 0)
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [681]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [682]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [683]:
%time clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [684]:
%time my_clf.fit(X_train, y_train)

Wall time: 15 ms


## Проверка качества работы на wine

In [685]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.7835497835497836

In [686]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.7835497835497836

## Подготовка данных Speed Dating Data 

In [687]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb

In [719]:
%%pycodestyle


df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')


def f(df):
    df = df.iloc[:, :97]
    df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
                 'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
                  'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o',
                  'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',
                  'like_o', 'prob_o', 'met_o', 'field', 'undergra',
                  'imprelig', 'imprace', 'from', 'zipcode', 'career', 'sports',
                  'tvsports', 'exercise', 'dining', 'museums',
                  'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
                  'theater', 'movies', 'concerts', 'music', 'shopping',
                  'yoga', 'expnum', 'wave'], axis=1)
    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
    df = pd.get_dummies(df, columns=['field_cd'],
                        prefix='field_cd', prefix_sep='=')
    df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',',
                                                          '').astype(np.float)
    df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
    df.loc[:, 'tuition'] =\
        df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
    df.loc[:, 'tuition'] = df.tuition.fillna(-999)
    df.loc[:, 'income'] =\
        df.loc[:, 'income'].str.replace(',', '').astype(np.float)
    df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
    df = df.dropna(subset=['date'])
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
    df = pd.get_dummies(df, columns=['career_c'],
                        prefix='career_c', prefix_sep='=')
    a = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
    df.loc[:, 'temp_totalsum'] = df.loc[:, a].sum(axis=1)
    df.loc[:, a] =\
        (df.loc[:, a].T / df.loc[:, 'temp_totalsum'].T).T * 100
    a = ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
    df.loc[:, 'temp_totalsum'] = df.loc[:, a].sum(axis=1)
    df.loc[:, a] =\
        (df.loc[:, a].T / df.loc[:, 'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)
    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)
    df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                .drop(['gender'], axis=1).dropna()
    df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                  .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                  .dropna()
    df_female.columns = df_female.columns + '_f'
    df_female = df_female.drop(['pid_f'], axis=1)
    df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
    df_pair = df_pair.drop(['iid', 'pid'], axis=1)
    X = df_pair.iloc[:, 1:].values
    y = df_pair.iloc[:, 0].values
    return X, y, df_pair

In [689]:
X,y,A = f(df)

In [690]:
A.head()

,match,int_corr,samerace,age,mn_sat,tuition,race,income,goal,date,...,career_c=8.0_f,career_c=9.0_f,career_c=10.0_f,career_c=11.0_f,career_c=12.0_f,career_c=13.0_f,career_c=14.0_f,career_c=15.0_f,career_c=16.0_f,career_c=17.0_f
1871,0,0.16,0,26.0,-999.0,-999.0,4.0,78193.0,1.0,4.0,...,0,0,0,0,0,0,0,0,0,0
1876,0,0.34,1,32.0,-999.0,-999.0,2.0,63351.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0
1881,0,0.52,0,37.0,-999.0,-999.0,4.0,-999.0,1.0,6.0,...,0,0,0,0,0,0,0,0,0,0
1886,1,0.13,1,29.0,-999.0,-999.0,2.0,-999.0,1.0,5.0,...,0,0,0,0,0,0,0,0,0,0
1891,0,0.05,0,28.0,-999.0,-999.0,3.0,-999.0,1.0,6.0,...,0,0,0,0,0,0,0,0,0,0


In [691]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [692]:
%time clf.fit(X_train, y_train)

Wall time: 82.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [693]:
%time my_clf.fit(X_train, y_train)

Wall time: 339 ms


## Проверка качества работы на Speed Dating Data

In [694]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5335978180014878

In [695]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.4818181818181818

## Задание 3

In [696]:
data = my_clf.feature_importances_ / np.sum(my_clf.feature_importances_)
My_feature = pd.Series(data=data, index=A.columns[1:]) 
My_feature.sort_values(ascending = False)[:10]

sinc4_1      0.161597
shar4_1      0.149219
go_out_f     0.142204
fun2_1_f     0.135497
exphappy     0.122001
sinc4_1_f    0.096291
income_f     0.080326
race_f       0.041148
tuition_f    0.038617
int_corr     0.033100
dtype: float64

In [697]:
Feature = pd.Series(data=clf.feature_importances_, index=A.columns[1:]) 
Feature.sort_values(ascending = False)[:10]

int_corr      0.070864
tuition_f     0.034682
age           0.033663
shar1_1       0.032284
shar2_1       0.029097
date          0.028563
intel1_1_f    0.026516
attr2_1       0.026027
attr3_1       0.025484
income_f      0.022571
dtype: float64

## Задание 4

In [724]:
clf = RandomForestClassifier(n_jobs=7) 
params = { 
"max_features": ['auto','sqrt','log2'], 
"max_depth": [i for i in range(1, 20)], 
"min_impurity_decrease": [i / 1000 for i in range(1, 20)] 
} 
A = RandomizedSearchCV(clf, 
param_distributions=params, n_iter=30) 
A.fit(X, y) 
print(A.best_params_)

{'min_impurity_decrease': 0.001, 'max_features': 'log2', 'max_depth': 8}
